In [206]:
#!/usr/bin/env python
import os
from witb.utils import ioutils as io
from witb.globals import DATA_DIR
import time  
import nltk
import csv
import re, string
import collections
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime

In [321]:
stop_words = set(stopwords.words('english'))

In [390]:
DATA_DIR="/home/sasha/Documents/MilaPostDoc/Python/CommonCrawl"

In [301]:
blacklistcsv='wordlists/porn_blacklist_filtered.csv'

In [332]:
bigrams='wordlists/porn_bigrams_filtered.csv'

In [333]:
bigramlist = []
with open(bigrams, newline='') as inputfile:
    for row in csv.reader(inputfile):
        bigramlist.append(tuple(row[0].split()))
print(len(bigramlist))

748


In [299]:
bigramlist

[('hairy', 'pussy'),
 ('wet', 'pussy'),
 ('black', 'pussy'),
 ('tight', 'pussy'),
 ('pussy', 'porn'),
 ('sex', 'videos'),
 ('pussy', 'sex'),
 ('shaved', 'pussy'),
 ('girl', 'pussy'),
 ('pussy', 'pics'),
 ('eating', 'pussy'),
 ('pussy', 'licking'),
 ('her', 'pussy'),
 ('pussy', 'free'),
 ('licking', 'pussy'),
 ('pink', 'pussy'),
 ('teen', 'pussy'),
 ('big', 'pussy'),
 ('pussy', 'fuck'),
 ('eat', 'pussy'),
 ('my', 'pussy'),
 ('naked', 'pussy'),
 ('fat', 'pussy'),
 ('pussy', 'and'),
 ('girl', 'sex'),
 ('having', 'sex'),
 ('lesbian', 'pussy'),
 ('porn', 'movies'),
 ('fucking', 'pussy'),
 ('pussy', 'videos'),
 ('sex', 'pussy'),
 ('pussy', 'video'),
 ('skinny', 'teen'),
 ('girl', 'porn'),
 ('in', 'pussy'),
 ('porn', 'videos'),
 ('white', 'pussy'),
 ('free', 'porn'),
 ('sex', 'video'),
 ('and', 'pussy'),
 ('sex', 'with'),
 ('anal', 'sex'),
 ('sex', 'movies'),
 ('pussy', 'pictures'),
 ('little', 'pussy'),
 ('pussy', 'lips'),
 ('girls', 'fucking'),
 ('pussy', 'fucked'),
 ('pussy', 'eating'),
 (

In [302]:
blacklist = []
with open(blacklistcsv, newline='') as inputfile:
    for row in csv.reader(inputfile):
        blacklist.append(row[0])
print(len(blacklist))

3568


## Working with raw CC data

In [346]:
filenames = os.listdir(DATA_DIR)
filename = os.path.join(DATA_DIR, filenames[0])

data = io.get_entries(io.read_wet_file(filename))
parsed_data = io.parse_data(data)  # Generator.

docs = []
start_time = time.time()
for doc in parsed_data:
    docs.append(doc)

### Simple word search

In [167]:
start = time.time()
pornsites={}
for doc in docs:
    pwds=len([w for w in word_tokenize(doc.content) if w in blacklist])
    pornsites[doc.url]=pwds
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

KeyboardInterrupt: 

### Bigram search (counts only)

In [367]:
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(doc.content) if w not in stop_words]
    bigrams=[b for b in nltk.bigrams(word_tokenize(str(' '.join(clean))))]
    match = set(bigrams).intersection(bigramlist)
    pornsites[doc.url]=len(match)
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

Number of pages counted:  16354
Duration:  0:01:44.870245


### Bigram search with counts and list of tuples found (for debugging)

In [ ]:
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(doc.content) if w not in stop_words]
    bigrams=[b for b in nltk.bigrams(word_tokenize(str(' '.join(clean))))]
    match = set(bigrams).intersection(bigramlist)
    pornsites[doc.url]=[len(match), str(match)]
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

## Working with JSON texts from CC_Net

In [ ]:
#Json processing
import json

docs = []
for line in open('/home/sasha/Documents/MilaPostDoc/Python/CommonCrawl/CCnet/en_tail_0000.json', 'r'):
    docs.append(json.loads(line))

In [ ]:
docs[0].keys()

In [ ]:
perpsort= sorted(docs, key=lambda k: k['perplexity'])

In [ ]:
for perp in perpsort[:500]:
    print(perp['perplexity'])
    print(perp['raw_content'][:1000])

In [398]:
#Bigram search with counts for clean text
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(doc['raw_content']) if w not in stop_words]
    bigrams=[b for b in nltk.bigrams(word_tokenize(str(' '.join(clean))))]
    match = set(bigrams).intersection(bigramlist)
    pornsites[doc['url']]=len(match)
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

Number of pages counted:  978
Duration:  0:00:09.508803


In [ ]:
#Bigram search with counts and list of tuples found (for debugging)
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(doc['raw_content']) if w not in stop_words]
    bigrams=[b for b in nltk.bigrams(word_tokenize(str(' '.join(clean))))]
    match = set(bigrams).intersection(bigramlist)
    pornsites[doc['url']]=[len(match), str(match)]
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

In [416]:
#Bigram search with counts and list of tuples found (for debugging)
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(doc.content) if w not in stop_words]
    bigrams=[b for b in nltk.bigrams(word_tokenize(str(' '.join(clean))))]
    match = set(bigrams).intersection(bigramlist)
    pornsites[doc.url]=[len(match), str(match)]
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

Number of pages counted:  13181
Duration:  0:01:11.796793


### Counting and sorting results

In [417]:
{k: v for k, v in sorted(pornsites.items(), key=lambda item: item[1], reverse=True)}

{'http://adultmovietop100.com/free-porn-eat-my-pussy-first-date-sex-vids-black-and-white-and-sex-full-movie-beresfield-music-and-dating': [241,
  "{('sex', 'xxx'), ('mobile', 'porn'), ('sexy', 'teen'), ('free', 'lesbian'), ('sexy', 'women'), ('hardcore', 'sex'), ('big', 'cock'), ('porn', 'picture'), ('sex', 'video'), ('tight', 'pussy'), ('hot', 'sex'), ('older', 'women'), ('porn', 'sex'), ('pussy', 'video'), ('cum', 'on'), ('women', 'sex'), ('porn', 'big'), ('sexy', 'hot'), ('asian', 'sex'), ('naked', 'girls'), ('porn', 'black'), ('sex', 'on'), ('amateur', 'sex'), ('sex', 'teen'), ('asian', 'girls'), ('mature', 'porn'), ('big', 'dicks'), ('lesbian', 'sex'), ('sex', 'pic'), ('pussy', 'women'), ('chubby', 'teen'), ('sexy', 'porn'), ('hot', 'blonde'), ('best', 'porn'), ('nude', 'girls'), ('lesbian', 'girls'), ('pussy', 'cum'), ('petite', 'teen'), ('sex', 'free'), ('mature', 'pussy'), ('shows', 'pussy'), ('nude', 'pic'), ('fuck', 'my'), ('pussy', 'free'), ('free', 'nude'), ('tube', 'sex'),

In [419]:
len({k: v for k, v in pornsites.items() if v >= 1})

374